In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Project ANNa: Spiking Neural Network (SNN) Visualization\n",
    "\n",
    "**Objective:** This notebook provides a look into the internal dynamics of the core SNNs running on the Loihi 2 hardware. \n",
    "\n",
    "While the `1_data_analysis.ipynb` notebook focuses on the high-level performance of the system (its output), this notebook visualizes the *process*—how the neural networks encode and process information. This is crucial for debugging, tuning, and understanding the bio-inspired computation.\n",
    "\n",
    "We will visualize two key concepts:\n",
    "1.  **Continuous Attractor Network:** The \"bump\" of neural activity that represents the robot's heading in the `AntBot SNN`.\n",
    "2.  **Hexagonal Grid Cell Firing:** The spatial firing pattern of a single neuron in the `GridCore SNN` as the robot explores an environment."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "--- \n",
    "### 1. Setup and Library Imports"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from scipy.ndimage import gaussian_filter1d\n",
    "\n",
    "# Configure plots for better readability and scientific presentation\n",
    "plt.style.use('seaborn-v0_8-whitegrid')\n",
    "plt.rcParams.update({\n",
    "    'font.size': 14,\n",
    "    'axes.labelsize': 16,\n",
    "    'axes.titlesize': 18,\n",
    "    'xtick.labelsize': 12,\n",
    "    'ytick.labelsize': 12,\n",
    "    'legend.fontsize': 14,\n",
    "    'figure.figsize': (12, 8)\n",
    "})"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "--- \n",
    "### 2. Visualization of AntBot SNN: Continuous Attractor Network\n",
    "\n",
    "A continuous attractor network is a ring of neurons that can sustain a localized \"bump\" of activity. The position of this bump represents a continuous variable, like the robot's heading (0-360 degrees). When the robot turns, an external input (from the IMU) pushes the bump to a new location, where it remains stable until the next movement. This provides a robust, short-term memory of the robot's orientation.\n",
    "\n",
    "We will simulate the activity of this network over time as the robot performs a series of turns."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def simulate_attractor_network(num_neurons=128, duration_steps=500):\n",
    "    \"\"\"Simulates the activity of a 1D ring attractor network for heading.\"\"\"\n",
    "    # Initialize neural activities\n",
    "    activity = np.zeros((num_neurons, duration_steps))\n",
    "    current_heading_idx = num_neurons // 4  # Start at 90 degrees\n",
    "    \n",
    "    # Define the shape of the activity bump (a Gaussian)\n",
    "    neuron_indices = np.arange(num_neurons)\n",
    "    bump_width = num_neurons / 10.0\n",
    "    \n",
    "    # Simulate inputs corresponding to turns\n",
    "    inputs = np.zeros(duration_steps)\n",
    "    inputs[100:150] = 1   # Turn right\n",
    "    inputs[250:350] = -1  # Turn left\n",
    "    inputs[400:425] = 2   # Turn right fast\n",
    "    \n",
    "    for t in range(1, duration_steps):\n",
    "        # Update heading based on input\n",
    "        current_heading_idx = (current_heading_idx + inputs[t]) % num_neurons\n",
    "        \n",
    "        # Calculate the distance of each neuron from the current heading (with wrap-around)\n",
    "        distances = np.abs(neuron_indices - current_heading_idx)\n",
    "        distances = np.minimum(distances, num_neurons - distances)\n",
    "        \n",
    "        # Generate the Gaussian bump of activity\n",
    "        bump_activity = np.exp(-0.5 * (distances / bump_width)**2)\n",
    "        activity[:, t] = bump_activity + np.random.normal(0, 0.05, num_neurons)\n",
    "        \n",
    "    return activity\n",
    "\n",
    "# Run the simulation\n",
    "attractor_activity = simulate_attractor_network()\n",
    "\n",
    "# Visualize the activity as a heatmap\n",
    "plt.figure(figsize=(15, 8))\n",
    "im = plt.imshow(attractor_activity, aspect='auto', cmap='viridis', origin='lower')\n",
    "\n",
    "cbar = plt.colorbar(im)\n",
    "cbar.set_label('Neural Activity (Firing Rate)')\n",
    "plt.title('AntBot SNN: Continuous Attractor Network for Heading')\n",
    "plt.xlabel('Time Steps')\n",
    "plt.ylabel('Neuron Index (Represents Heading)')\n",
    "plt.yticks(ticks=[0, 32, 64, 96, 127], labels=['0°', '90°', '180°', '270°', '360°'])\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**Interpretation:** The bright yellow band shows the stable \"bump\" of activity. You can see how it remains in one position (maintaining memory of the heading) and then smoothly shifts to a new position when an input corresponding to a turn is applied. This visualizes the core mechanism behind the AntBot SNN's robust odometry."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "--- \n",
    "### 3. Visualization of GridCore SNN: Hexagonal Firing Fields\n",
    "\n",
    "Grid cells in the mammalian brain fire in a periodic hexagonal pattern as an animal explores a 2D space. The brain combines the signals from many grid cells (with different scales and orientations) to create a universal metric for space, effectively a coordinate system. Our `GridCore SNN` replicates this.\n",
    "\n",
    "We will simulate a robot's path and the firing rate of a single grid cell to reveal this beautiful and iconic hexagonal pattern."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_grid_cell_firing_rate(x, y, scale=5, orientation=np.pi/6):\n",
    "    \"\"\"Calculates the firing rate of a single grid cell at a given (x,y) position.\"\"\"\n",
    "    # Define the three vectors for the hexagonal grid, rotated by the orientation\n",
    "    k1 = np.array([np.cos(orientation), np.sin(orientation)])\n",
    "    k2 = np.array([np.cos(orientation + np.pi/3), np.sin(orientation + np.pi/3)])\n",
    "    k3 = np.array([np.cos(orientation + 2*np.pi/3), np.sin(orientation + 2*np.pi/3)])\n",
    "    \n",
    "    # The firing rate is the sum of three cosines\n",
    "    pos = np.array([x, y]) / scale\n",
    "    rate = (np.cos(np.dot(k1, pos)) + np.cos(np.dot(k2, pos)) + np.cos(np.dot(k3, pos))) / 3.0\n",
    "    \n",
    "    # Normalize to be mostly positive\n",
    "    return np.clip(rate, 0, 1)\n",
    "\n",
    "# Simulate a random exploration path for the robot\n",
    "path_length = 2000\n",
    "path = np.zeros((path_length, 2))\n",
    "for i in range(1, path_length):\n",
    "    step = np.random.randn(2) * 0.2\n",
    "    path[i] = path[i-1] + step\n",
    "\n",
    "# Calculate the firing rate of our grid cell along the path\n",
    "firing_rates = np.array([get_grid_cell_firing_rate(p[0], p[1]) for p in path])\n",
    "\n",
    "# Visualize the path and firing fields\n",
    "plt.figure(figsize=(12, 12))\n",
    "sc = plt.scatter(path[:, 0], path[:, 1], c=firing_rates, cmap='plasma', s=30, alpha=0.7)\n",
    "\n",
    "cbar = plt.colorbar(sc)\n",
    "cbar.set_label('Grid Cell Firing Rate')\n",
    "plt.title('GridCore SNN: Firing Field of a Single Grid Cell')\n",
    "plt.xlabel('X Position (meters)')\n",
    "plt.ylabel('Y Position (meters)')\n",
    "plt.axis('equal')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "**Interpretation:** The plot shows the robot's exploratory path. The color of the path indicates the firing rate of the simulated grid cell. The bright yellow spots, where the neuron fired most intensely, are arranged in a clear hexagonal lattice. This demonstrates that the `GridCore SNN` is successfully encoding the 2D space with a periodic, metric structure, which is the foundation for creating a cognitive map."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "--- \n",
    "### 4. Conclusion\n",
    "\n",
    "These visualizations provide strong evidence that the SNNs in Project ANNa are not just black boxes, but are implementing specific, powerful computational strategies observed in biology.\n",
    "\n",
    "- The **attractor network** provides a stable, low-power memory for state variables like heading.\n",
    "- The **grid cell network** provides a robust, metric foundation for mapping and understanding spatial relationships.\n",
    "\n",
    "Together, these visualizations confirm that the project is on the right track to building a truly bio-inspired and intelligent navigation system."
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}